In [1]:
from vncorenlp import VnCoreNLP

In [2]:
import numpy as np

In [3]:
def preTreatment(sentences, length):
    punctuations = []     # Dấu câu
    with open("punctuation.txt", "r", encoding = "utf-8") as data:
        for line in data:
            punctuations.append(line.strip())
    data.close()
    
    stopWords = []
    with open("vietnamese-stopwords-dash.txt", "r", encoding = "utf-8") as data:
        for line in data:
            stopWords.append(line.strip())
    data.close()
    
    for i in range(0, length):
        for j in range(0, len(sentences[i])):
            sentences[i][j] = sentences[i][j].lower()
        #print(sentences[i])
        
    # Loại bỏ dấu câu và stop words
    for i in range(0, length):
        sentences[i] = list(set(sentences[i]) - set(punctuations) - set(stopWords))
        #print(sentences[i])

In [4]:
def adjacentMatrix(sentences, length):
    # Tạo ma trận kề của đồ thị có trọng số
    matrix = np.zeros((length, length))
    for i in range(0, length):
        for j in range(0, length):
            if i != j:
                jaccard = len(set(sentences[i]) & set(sentences[j])) / len(set(sentences[i]) | set(sentences[j]))
                if jaccard != 0:
                    matrix[i][j] = jaccard
    #for i in range(0, length):
        #for j in range(0, length):
            #print(matrix[i][j], end="\t")
        #print()
    return matrix

In [5]:
def textRank(matrix, length):
    # sumOut là vector trong đó sumOut[i] là tổng trọng số của các link out của matrix[i]
    sumOut = np.zeros(length)
    for i in range(0, length):
        for j in range(0, length):
            if matrix[i][j] != 0:
                sumOut[i] += matrix[i][j]
    #print(sumOut)
    
    WS = np.ones(length)        # vector điểm của các đỉnh ở lần tính thứ n
    preWS = np.ones(length)     # vector điểm của các đỉnh ở lần tính thứ n - 1
    
    d = 0.85
    x = 0
    
    # Áp dụng thuật toán ranking trên đồ thị có trọng số
    while x < 100:
        for i in range(0, length):
            sum = 0
            for j in range(0, length):
                if matrix[i][j] != 0:
                    sum += (matrix[j][i] / sumOut[j]) * preWS[j]
            WS[i] = (1 - d) + d * sum
        for i in range(0, length):
            preWS[i] = WS[i]
        #print("WS lần thứ:", x)
        #print(WS)
        x += 1
    return WS

In [6]:
def sentencesExtraction(WS, word_segmented_text, length):
    # Tìm đỉnh có điểm cao nhất
    dictWS = {}
    for i in range(0, length):
        dictWS[WS[i]] = i
        
    mainSentences = []
    
    for i in sorted(dictWS.keys()):
        mainSentences.append(word_segmented_text[dictWS[i]])
    return mainSentences

In [7]:
def main():
    annotator = VnCoreNLP(r"C:\Users\admin\Desktop\kit\VnCoreNLP\VnCoreNLP-1.1.1.jar", annotators="wseg,pos,parse", max_heap_size='-Xmx2g')
    
    data = open("text.txt", "r", encoding="utf-8")
    text = data.read()
    #text = "Ông Nguyễn Khắc Chúc đang làm việc tại Đại học Quốc gia Hà Nội. Bà Lan, vợ ông Chúc, cũng làm việc tại đây."
    
    annotated_text = annotator.annotate(text)
    word_segmented_text = annotator.tokenize(text)
    
    sentences = word_segmented_text.copy()
    length = len(word_segmented_text)
    
    preTreatment(sentences, length)
    matrix = adjacentMatrix(sentences, length)
    WS = textRank(matrix, length)
    mainSentences = sentencesExtraction(WS, word_segmented_text, length)
    
    print(mainSentences[length - 1])

In [8]:
main()

['luật_sư', 'lê_trung_sơn', 'phân_tích', 'thêm', ',', 'khi', 'cho', 'ý_kiến', 'đồng_ý', 'với', 'nội_dung', 'tờ_trình', 'số', '835', ',', 'bị_cáo', 'hưng', 'chỉ', 'có', 'mong_muốn', 'đổi_mới', ',', 'tháo_gỡ', 'khó_khăn', 'cho', 'dự_án', '.']
